In [1]:
import sys
import os

import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pprint

from monai.networks.nets.densenet import DenseNet121, DenseNet169, DenseNet201, DenseNet264
from monai.networks.nets.efficientnet import EfficientNetBN
from monai.networks.nets.resnet import ResNet, resnet34, resnet50, resnet101, resnet152, resnet200

from warnings import filterwarnings
filterwarnings("ignore")

sys.path.append(os.path.join(str(os.path.abspath('')), "..", "..", ".."))

from src.train_one_epoch import train_one_epoch
from src.get_data_loaders import prepare_train_valid_dataloader
from src.validate_func import valid_func

In [2]:
class CFG:
    debug = False # change this to run on full data
    
    image_size = 256
    folds = [0, 1, 2, 3, 4]
    
    kernel_type = "resnet34"
    
    train_batch_size = 6
    valid_batch_size = 24
    
    num_images = 64
    mri_type = 'T2w'
    
    init_lr = 1e-4
    weight_decay=0
    
    n_epochs = 20
    num_workers = 4

    use_amp=True
    early_stop = 5

    data_dir = PATH_TO_DATA # !!! DEFINE "PATH_TO_DATA" on your local machine
    model_dir = f'weights/'
    seed=12345
    

In [3]:
results_dir = CFG.mri_type + "_weights/"

In [4]:
! mkdir $results_dir

mkdir: cannot create directory ‘T2w_weights/’: File exists


In [5]:
df_train = pd.read_csv('../../crossval/train_df_folds.csv')
if CFG.debug:
    df_train = df_train.sample(frac=0.1)
df_train.head()

,BraTS21ID,MGMT_value,fold
0,0,1,2
1,2,1,1
2,3,0,1
3,5,1,4
4,6,1,1


In [6]:
criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
for fold in CFG.folds:
    train_loader, valid_loader = prepare_train_valid_dataloader(
        df=df_train, fold=fold, num_images=CFG.num_images,
        img_size=CFG.image_size, data_directory=CFG.data_dir, mri_type=CFG.mri_type,
        train_batch_size=CFG.train_batch_size, valid_batch_size=CFG.valid_batch_size,
        num_workers=CFG.num_workers
    )
    
#     model = DenseNet121(spatial_dims=3, in_channels=1, out_channels=1).to(device)
#     model = EfficientNetBN(spatial_dims=3, in_channels=1, num_classes=1, model_name="efficientnet-b0").to(device)
    model = resnet34(spatial_dims=3, n_input_channels=1, num_classes=1).to(device)

    optimizer = optim.Adam(model.parameters(), lr=CFG.init_lr, weight_decay=CFG.weight_decay)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

#     scheduler = ReduceLROnPlateau(
#         optimizer, mode='min', patience=1, min_lr=1e-6, factor=0.1, verbose=True, eps=1e-8
#     )

    num_epochs = CFG.n_epochs

    print("-----------------------------------------------------------------------------------------------------")
    print("                                        FOLD: ", fold)
    print("-----------------------------------------------------------------------------------------------------")
    
    roc_auc_max = 0.0
    loss_min = 99999
    ap_max = 0.0
    not_improving = 0
    metrics_list = list()
    
    for epoch in range(CFG.n_epochs):
        
        loss_train, roc_auc_train = train_one_epoch(
            model, device, criterion, optimizer, train_loader, CFG.use_amp)
        
        loss_valid, roc_auc_valid = valid_func(
            model, device, criterion, valid_loader)
        
        scheduler.step()
        
#         scheduler.step(loss_valid)
        
        metrics_dictionary = {}
        metrics_dictionary['epoch'] = epoch
        metrics_dictionary['loss_train'] = loss_train
        metrics_dictionary['loss_valid'] = loss_valid
        metrics_dictionary['roc_auc_train'] = roc_auc_train
        metrics_dictionary['roc_auc_valid'] = roc_auc_valid
        metrics_dictionary['fold'] = fold
        pprint.pprint(metrics_dictionary)
        metrics_list.append(metrics_dictionary)
        
        not_improving += 1
        if roc_auc_valid > roc_auc_max:
            print(f'roc_auc_max ({roc_auc_max:.6f} --> {roc_auc_valid:.6f}). Saving model ...')
            torch.save(model.state_dict(), f'{results_dir}{CFG.kernel_type}_fold{fold}_best_AUC_{CFG.mri_type}_mri_type.pth')
            roc_auc_max = roc_auc_valid
            not_improving = 0

        if loss_valid < loss_min:
            loss_min = loss_valid
            print(f'loss_min ({loss_min:.6f} --> {loss_valid:.6f}). Saving model ...')
            torch.save(model.state_dict(), f'{results_dir}{CFG.kernel_type}_fold{fold}_best_loss_{CFG.mri_type}_mri_type.pth')

            
        if not_improving == CFG.early_stop:
            print('Early Stopping...')
            break

    
    metrics = pd.DataFrame(metrics_list)
    metrics.to_csv(f'{results_dir}{CFG.kernel_type}_fold{fold}_final.csv', index=False)
    torch.save(model.state_dict(), f'{results_dir}{CFG.kernel_type}_fold{fold}_final_{CFG.mri_type}_mri_type.pth')


-----------------------------------------------------------------------------------------------------
                                        FOLD:  0
-----------------------------------------------------------------------------------------------------


loss: 0.75587, total_loss: 0.69554: 100%|██████████████| 5/5 [01:29<00:00, 17.93s/it]


{'epoch': 0,
 'fold': 0,
 'loss_train': 0.7583113924050943,
 'loss_valid': 0.6955398797988892,
 'roc_auc_train': 0.524608708552778,
 'roc_auc_valid': 0.5879765395894428}
roc_auc_max (0.000000 --> 0.587977). Saving model ...
loss_min (0.695540 --> 0.695540). Saving model ...


loss: 0.82749, total_loss: 0.84316: 100%|██████████████| 5/5 [00:19<00:00,  3.96s/it]


{'epoch': 1,
 'fold': 0,
 'loss_train': 0.695853475576792,
 'loss_valid': 0.8431619167327881,
 'roc_auc_train': 0.5464728135894963,
 'roc_auc_valid': 0.5428152492668622}


...

{'epoch': 4,
 'fold': 4,
 'loss_train': 0.6830070717976644,
 'loss_valid': 0.6700934767723083,
 'roc_auc_train': 0.589768214978299,
 'roc_auc_valid': 0.6205663189269746}
loss_min (0.670093 --> 0.670093). Saving model ...


loss: 0.69891, total_loss: 0.69786: 100%|██████████████| 5/5 [00:17<00:00,  3.59s/it]


{'epoch': 5,
 'fold': 4,
 'loss_train': 0.6910755748932178,
 'loss_valid': 0.6978554606437684,
 'roc_auc_train': 0.5690737833594977,
 'roc_auc_valid': 0.5684053651266765}


loss: 0.70782, total_loss: 0.69808: 100%|██████████████| 5/5 [00:17<00:00,  3.57s/it]


{'epoch': 6,
 'fold': 4,
 'loss_train': 0.6717615169592392,
 'loss_valid': 0.6980847835540771,
 'roc_auc_train': 0.6069720195770616,
 'roc_auc_valid': 0.5463487332339791}
Early Stopping...
